In [1]:
#####Bi-level model for Rare temporal pattern detection####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import math
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(98537)

In [2]:
####データの発生####
##データの設定
k = 2   #混合数
m0 = 5   #正常状態での潜在変数数
m1 = 2   #異常状態での潜在変数数
hh = 5000   #ユーザー数
pt = np.random.poisson(np.random.gamma(50, 1/0.2, hh), hh)   #系列数
hhpt = np.sum(pt)   #総レコード数

In [3]:
##IDとインデックスを設定
#IDを設定
user_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスを設定
index = np.arange(hhpt)
user_list = [i for i in range(hh)]
user_vec = [i for i in range(hh)]
for i in range(hh):
    user_list[i] = index[user_id==i]
    user_vec[i] = np.repeat(1, pt[i])

In [11]:
##潜在パラメータとユーザーの異常状態を生成
#潜在変数のパラメータ
alpha = np.repeat(0.5, m0*m0).reshape(m0, m0) + np.diag(np.repeat(1.0, m0))
pi = 0.03   #ユーザーの混合率
gamma = np.array([0.05, 0.6])   #正常状態と異常状態の切換率
delta = 0.5   #異常状態の混合率
omega = np.append(np.random.dirichlet(np.repeat(2.5, m0), 1).reshape(-1), 0)   #正常状態の推移率の初期値
theta = np.zeros((m0+1, m0+1))
for j in range(m0):
    theta[j, ] = np.append(np.random.dirichlet(alpha[j, ], 1), 0)
theta[m0, ] = np.append(np.random.dirichlet(np.repeat(1.0, m0), 1), 0)
pit = pi; gammat = gamma; deltat = delta
omegat = omega; thetat = theta.copy()
    
#ユーザーの潜在変数を生成
Z = np.random.binomial(1, pi, hh)
index_z = np.array(np.where(Z==1)[0], dtype="int")
n = index_z.shape[0]

#異常状態の潜在変数を生成
A = np.repeat(0, hh)
A[index_z] = np.random.binomial(1, delta, n) + 1

In [12]:
##ユーザーごとに系列の異常状態を生成
#データの格納用リスト
S0_list = [i for i in range(hh)]
S_list = [i for i in range(hh)]

for i in range(hh):
    #系列の格納用配列
    s0 = np.array(np.zeros((pt[i], m0+1)), dtype="int")
    s = np.repeat(0, pt[i])

    #正常ユーザーの潜在変数を生成
    if Z[i]==0:
        for j in range(pt[i]):
            if j==0:
                s0[j, ] = np.random.multinomial(1, omega, 1)
            else:
                index = np.argmax(s0[j-1, ])
                s0[j, ] = np.random.multinomial(1, theta[index, ], 1)

    #異常ユーザーの潜在変数を生成
    else:
        for j in range(pt[i]):
            if j==0:
                s[j] = np.random.binomial(1, gamma[0], 1)
                if s[j]==0:
                    s0[j, ] = np.random.multinomial(1, omega, 1)
                else:
                    s0[j, m0] = 1
            else:
                if s[j-1]==0:
                    s[j] = np.random.binomial(1, gamma[0], 1)
                else:
                    s[j] = np.random.binomial(1, gamma[1], 1)
                if s[j]==0:
                    index = np.argmax(s0[j-1, ])
                    s0[j, ] = np.random.multinomial(1, theta[index, ], 1)
                else:
                    s0[j, m0] = 1
                    
    #生成したデータの格納
    S0_list[i] = s0
    S_list[i] = s

In [13]:
#リストを変換
S = np.array(list(itertools.chain(*[S_list[i] for i in range(hh)])))
S0 = np.array(list(itertools.chain(*[S0_list[i] for i in range(hh)])))
print(np.sum(S0, axis=0))

[165661  86561 487323 215068 288276   4580]


In [15]:
##潜在変数からデータを生成
#入力変数を生成



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])